In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C:\\SPB_Data\\IIT Kanpur Applied Machine Learning and Data Science\\Kaggle Project\\Sentiment Analysis on Twitter\\train.txt'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#importing all the libraries

import sys , os , re , csv , codecs , numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Input,LSTM,Embedding,Dropout,Activation,BatchNormalization
from keras.layers import Bidirectional,GlobalMaxPool1D,GlobalAvgPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints , optimizers, layers
from keras.utils import to_categorical


Using TensorFlow backend.


In [3]:
#getting the train data
train = pd.read_csv('C:\\SPB_Data\\IIT Kanpur Applied Machine Learning and Data Science\\Kaggle Project\\Sentiment Analysis on Twitter\\train.txt')
#loading the test data
test = pd.read_csv('C:\\SPB_Data\\IIT Kanpur Applied Machine Learning and Data Science\\Kaggle Project\\Sentiment Analysis on Twitter\\test_samples.txt')
#diplay first 5 rows of train
train.head()
#one hot encoding the labels
df = pd.concat([train,pd.get_dummies(train['sentiment'])],axis=1)
#df.head()
train_data = df['tweet_text']
#train_data.head()

In [4]:
test_data = test['tweet_text']
test_data.head()

0    @jjuueellzz down in the Atlantic city, ventnor...
1    Musical awareness: Great Big Beautiful Tomorro...
2    On Radio786 100.4fm 7:10 Fri Oct 19 Labour ana...
3    Kapan sih lo ngebuktiin,jan ngomong doang Susa...
4    Excuse the connectivity of this live stream, f...
Name: tweet_text, dtype: object

In [5]:
#creating the array of labels in serial with their respective texts
classes = ['neutral' , 'negative' , 'positive']
y = df[classes].values
y

array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0]], dtype=uint8)

In [6]:
#checking for null values in train and test data
train.isnull().any()
test.isnull().any()

tweet_id      False
tweet_text    False
dtype: bool

In [7]:
#configuration  parameters
LATENT_DIM_DECODER = 400
BATCH_SIZE =128
EPOCHS = 20
LATENT_DIM = 400
NUM_SAMPLES = 10000
MAX_SEQUENCE_LEN = 1000
MAX_NUM_WORDS = 100000
EMBEDDING_DIM = 300


In [8]:
#NLTK python library for preprocessing
import nltk
#nltk.download('wordnet')
#for tokenization
from nltk.tokenize import RegexpTokenizer
#for stemming
from nltk.stem import WordNetLemmatizer,PorterStemmer
#for removing stopwords
from nltk.corpus import stopwords
#importing regex library of python
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
#function for performing all preproccing steps at once
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens]#  if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

#make a dataframe of preprocessed text
df['cleanText']=train_data.map(lambda s:preprocess(s)) 

In [9]:
test['clean_text']=test['tweet_text'].map(lambda s:preprocess(s))
test_final = test['clean_text']
test_final

0       jjuueellzz down in the atlantic city ventnor m...
1       musical awareness great big beautiful tomorrow...
2       on radio fm fri oct labour analyst shawn hatti...
3       kapan sih lo ngebuktiin jan ngomong doang susa...
4       excuse the connectivity of this live stream fr...
5       show your love for your local field it might w...
6       milton on bolton wanderers v leeds united sat ...
7       firecore can you tell me when an update for th...
8       heavensbasement the crown filthy mcnastys katy...
9       uncover the eternal city return flights to rom...
10      my cre blog oklahoma per square foot returns t...
11      going to helsinki tomorrow or on the day after...
12      trey burke has been suspended for the northern...
13      w o w wednesday marni lands this lumberjack ve...
14      activists in deir ezzor captured this image of...
15      karaotr you will appreciate this sunday brunch...
16      no school for me and nat tomorrow sucks for yo...
17      join m

In [10]:
#breaking the sentence into unique words/tokens
#expecting max tokens to be 20k
train_final = df['cleanText']
max_feat=40000
#tokenize sentence into list of words
tokenizer = Tokenizer(num_words=max_feat)#setting up tokenizer
#fiiting the tokenizer on out data
tokenizer.fit_on_texts(list(train_final))


In [11]:
train_final

0        gas by my house hit i um going to chapel hill ...
1        theo walcott is still shit uc watch rafa and j...
2        its not that i um a gsp fan uc i just hate nic...
3        iranian general says israel us iron dome can u...
4        tehran uc mon amour obama tried to establish t...
5        i sat through this whole movie just for harry ...
6        with j davlar th main rivals are team poland h...
7        talking about act us sat us uc deciding where ...
8        why is happy valentines day trending it us on ...
9        they may have a superbowl in dallas uc but dal...
10       im bringing the monster load of candy tomorrow...
11       apple software uc retail chiefs out in overhau...
12       oluoch victor_otti kunjand i just watched it s...
13       one of my best th graders kory was excited aft...
14       livewire nadal confirmed for mexican open in f...
15       msshelahy i didnt want to just pop up but yep ...
16       alyoup addictedhaley hmmmm november is an odd .

In [12]:
tokenizer2 = Tokenizer(num_words=max_feat)#setting up tokenizer
#fiiting the tokenizer on out data
tokenizer2.fit_on_texts(list(test_final))

In [13]:
#converting text into sequence of numbers to feed in neural network
sequence_train = tokenizer.texts_to_sequences(train_final)
sequence_test = tokenizer2.texts_to_sequences(test_final)
# get the word to index mapping for input language
word2idx_inputs = tokenizer.word_index
print('Found %s unique input tokens.' % len(word2idx_inputs))


Found 34302 unique input tokens.


In [15]:
#LOADING PRETRAINED WORD VECTORS
# store all the pre-trained word vectors
print('Loading word vectors...')
word2vec = {}
with open('/kaggle/input/glove.6B.300d.txt', encoding="utf8") as f:
    # is just a space-separated text file in the format:
    # word vec[0] vec[1] vec[2] ...
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))


Loading word vectors...


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/glove.6B.300d.txt'

In [16]:
#EMBEDDING MATRIX
# prepare embedding matrix of words for embedding layer
print('Filling pre-trained embeddings...')
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx_inputs.items():
    if(i < MAX_NUM_WORDS):
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all zeros.
            embedding_matrix[i] = embedding_vector

Filling pre-trained embeddings...


In [17]:
max_len = [len(s) for s in sequence_train]
print(max(max_len))

1074


In [18]:
#scaling all the sequences to a fixed length
#dimension of input to the layer should be constant
#scaling each comment sequence to a fixed length to 200
#comments smaller than 200 will be padded with zeros to make their length as 200
max_len=1000
#pad the train and text sequence to be of fixed length (in keras input in lstm should be of fixed length sequnece)
x_train=pad_sequences(sequence_train,maxlen=max_len)
x_test=pad_sequences(sequence_test,maxlen=max_len)

In [ ]:
# create embedding layer
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  input_length=max_len,
  trainable=True
)

Checking if we used right length of sequence padding for greater length of comments we have to take such length sequence padding that on deleting some information doesnt result in loss of important imformation

In [ ]:
len_words = [len(words) for words in sequence_train]
#distribution of sequence
plt.hist(len_words, bins = np.arange(0,400,10))
plt.show()
# we can see that most of the comments have [0,50]  words

In [ ]:
input = Input(shape=(max_len,))

In [ ]:
#feeding the output of previous layer to the embedding layer that converts 
#the sequences into vector representation to detect relevance and context 
#of a particular word
embed_layer =embedding_layer(input)

In [ ]:
#passing the previous output as input to the BI_LSTM layer
LSTM_layer = Bidirectional(LSTM(256, return_sequences=True, name='BI_lstm_layer'))(embed_layer)
sec_LSTM_layer = Bidirectional(LSTM(256, return_sequences=True, name='BI2_lstm_layer'))(LSTM_layer)
batchnorm = BatchNormalization()(sec_LSTM_layer)

In [ ]:
#dimension reduction using pooling layer
red_dim_layer = GlobalAvgPool1D()(batchnorm)

In [ ]:
##### adding dropout layer for better generalization
#setting value as 0.1 , which means 10$ of nodes will be randomly disabled
drop_layer = Dropout(0.55)(red_dim_layer)

In [ ]:
#densely connected layer
dense1 = Dense(128,activation='elu')(drop_layer)
batchnorm2 = BatchNormalization()(dense1)
dense2 = Dense(128,activation='elu')(batchnorm2)
batchnorm3 = BatchNormalization()(dense2)
dense3 = Dense(128,activation='elu')(batchnorm3)

In [ ]:
#adding another dropout layer
drop_layer2 = Dropout(0.55)(dense3)

In [ ]:
#adding the output dense layer with sigmoid activation to get result 
#3  classes as output
output_dense = Dense(3,activation='softmax')(drop_layer2)

In [ ]:
#connecting the inputs and outputs to create a model and compiling the model
from keras.optimizers import Adagrad,Adam,RMSprop
model = Model(inputs=input , outputs = output_dense)
model.compile(loss = 'categorical_crossentropy',
             optimizer = RMSprop(lr=0.001),
             metrics = ['accuracy'])
model.summary()

In [ ]:
#Fitting the model 
batch_size=64
epochs = 30
model.fit(x_train,y,batch_size=batch_size,epochs = epochs,validation_split=0.2)